We have learned to build micrograds and autograds from scratch. We used a scalar class called "Value". However, as a Neural Network grows in size, it is extremely computationally inefficient to work with scalars only. Thus we need to implement a tensor class which holds matrices of nxm dimensions.

In [2]:
import numpy as np

In [3]:
class tensor:
    def __init__(self, fromArray=np.zeros((2,2)), _children = (), _operation = ''):
        fromArray = fromArray if isinstance(fromArray, np.ndarray) else np.array(fromArray)
        assert len(fromArray.shape) == 2, "Only 2D Tensors or Scalar to 2D Supported!"
        self.matrix = fromArray
        self.rows = fromArray.shape[0]
        self.columns = fromArray.shape[1]
        self._prev = set(_children)
        self._operation = _operation


    def __repr__(self):
        return f"Tensor Values = {self.matrix}"
    
    @classmethod
    def zeros(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = np.zeros((rows, columns), dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def random(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = (np.random.rand(rows, columns)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def const(cls, rows, columns, constant=1, dtype = np.float32):
        t = tensor()
        t.matrix = (np.full((rows, columns), constant)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    
    def shape(self):
        return (self.rows, self.columns)
    
    def __add__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix + other.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix - other.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    def __rsub__(self, other):
        other = self.checkOther(other)
        out_matrix = other.matrix - self.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    
    def __mul__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix * other.matrix
        out = tensor(out_matrix, (self, other), '*')
        return out
    
    def __rmul__(self, other):
        return self * other

        
    def __matmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        assert other.shape()[0] == self.shape()[-1], "Dimension Unsupported for @"
        out_matrix = self.matrix @ other.matrix
        out = tensor(out_matrix, (self, other), '@')

        return out
    
    def transpose(self):
        out_matrix = self.matrix.transpose()
        out = tensor(out_matrix, (self, ), 'T')

        return out

    def checkOther(self, other):
        if isinstance(other, int | float):
            other = tensor.const(self.rows, self.columns, other)
        elif not isinstance(other, tensor):
            other = tensor(other)
        assert other.shape() == self.shape(), "Operand Tensor sizes dont match"

        return other
        

In [149]:
test1 = [[3,5],
        [2,1]]
test2 = [[2,2],
        [2,3]]
t1 = tensor(test1)
t2 = tensor(test2)

t1 @ t2
t1.transpose()

Tensor Values = [[3 2]
 [5 1]]

In [150]:
a = tensor([[1, 2]])
b = tensor([[3, 4]])
c = a + b
d = c * a

print(d._prev)     
print(c._prev)     
print(d._operation) 
print(c._operation) 

{Tensor Values = [[4 6]], Tensor Values = [[1 2]]}
{Tensor Values = [[3 4]], Tensor Values = [[1 2]]}
*
+


Lets save our current state of Tensor class as a checkpoint. Similar to Value class for micrograd and autograd, we have implemented operations and children tracking. We have successfully implemented the computation graph model for tensors. We can now move on to back propagation.

In [4]:
class tensor:
    def __init__(self, fromArray=np.zeros((2,2)), _children = (), _operation = ''):
        fromArray = fromArray if isinstance(fromArray, np.ndarray) else np.array(fromArray)
        assert len(fromArray.shape) == 2, "Only 2D Tensors or Scalar to 2D Supported!"
        self.matrix = fromArray
        self.rows = fromArray.shape[0]
        self.columns = fromArray.shape[1]
        self._prev = set(_children)
        self._operation = _operation
        self.grad = None


    def __repr__(self):
        return f"Tensor Values = {self.matrix}"
    
    @classmethod
    def zeros(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = np.zeros((rows, columns), dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def random(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = (np.random.rand(rows, columns)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def const(cls, rows, columns, constant=1, dtype = np.float32):
        t = tensor()
        t.matrix = (np.full((rows, columns), constant)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    
    def shape(self):
        return (self.rows, self.columns)
    
    def __add__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix + other.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix - other.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    def __rsub__(self, other):
        other = self.checkOther(other)
        out_matrix = other.matrix - self.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    

    def __mul__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix * other.matrix
        out = tensor(out_matrix, (self, other), '*')
        return out
    
    def __rmul__(self, other):
        return self * other

        
    def __matmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        assert other.shape()[0] == self.shape()[-1], "Dimension Unsupported for @"
        out_matrix = self.matrix @ other.matrix
        out = tensor(out_matrix, (self, other), '@')

        return out
    
    def transpose(self):
        out_matrix = self.matrix.transpose()
        out = tensor(out_matrix, (self, ), 'T')

        return out

    def checkOther(self, other):
        if isinstance(other, int | float):
            other = tensor.const(self.rows, self.columns, other)
        elif not isinstance(other, tensor):
            other = tensor(other)
        assert other.shape() == self.shape(), "Operand Tensor sizes dont match"

        return other
    
    def zero_grad(self):
        self.grad = None
        

In [155]:
t = tensor()
print(t.grad)

None


Now let us add the backward lamda functions for each operation. This would be a headache. Either you can first derive them on paper yourself based on the concepts of Jacobians, downstream/upstream/local gradients, tensor contaction etc. or use standard formula. The first one is recommended for sound understanding. For reference, there are notes present in this directory.

In [ ]:
class tensor:
    def __init__(self, fromArray=np.zeros((2,2)), _children = (), _operation = ''):
        fromArray = fromArray if isinstance(fromArray, np.ndarray) else np.array(fromArray)
        assert len(fromArray.shape) == 2, "Only 2D Tensors or Scalar to 2D Supported!"
        self.matrix = fromArray
        self.rows = fromArray.shape[0]
        self.columns = fromArray.shape[1]
        self._prev = set(_children)
        self._operation = _operation
        self._backward = lambda : None
        self.grad = None


    def __repr__(self):
        return f"Tensor Values = {self.matrix}"
    
    @classmethod
    def zeros(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = np.zeros((rows, columns), dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def random(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = (np.random.rand(rows, columns)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def const(cls, rows, columns, constant=1, dtype = np.float32):
        t = tensor()
        t.matrix = (np.full((rows, columns), constant)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    
    def shape(self):
        return (self.rows, self.columns)
    
    def __add__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix + other.matrix

        def _backward():
            self.grad = np.zeros_like(out.grad) if self.grad is None else self.grad
            other.grad = np.zeros_like(out.grad) if other.grad is None else other.grad
            self.grad += out.grad #Derivation in the notes. 
            other.grad += out.grad
        out = tensor(out_matrix, (self, other), '+')
        out._backward = _backward
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        other = self.checkOther(other)
        return self + (-1 * other)
    
    
    def __rsub__(self, other):
        other = self.checkOther(other)
        return other + (-1 * other)
    

    def __mul__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix * other.matrix
        def _backward():
            self.grad = np.zeros_like(out.grad) if self.grad is None else self.grad
            other.grad = np.zeros_like(out.grad) if other.grad is None else other.grad
            self.grad += out.grad * other.matrix #Derivation in the notes. 
            other.grad += out.grad * self.matrix

        out = tensor(out_matrix, (self, other), '*')
        out._backward = _backward
        return out
    
    def __rmul__(self, other):
        return self * other

        
    def __matmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        assert other.shape()[0] == self.shape()[-1], "Dimension Unsupported for @"
        out_matrix = self.matrix @ other.matrix
        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            other.grad = np.zeros_like(other.matrix) if other.grad is None else other.grad
            self.grad += out.grad @ (other.matrix).transpose()#Derivation in the notes.
            other.grad += (self.matrix).transpose() @ out.grad 
        out = tensor(out_matrix, (self, other), '@')
        out._backward = _backward
        return out
    
    def transpose(self):
        out_matrix = self.matrix.transpose()
        
        def _backward():
            self.grad = np.zeros_like(out.grad.transpose()) if self.grad is None else self.grad
            self.grad += (out.grad).transpose() #Not in note, but can be derived similarly.

        out = tensor(out_matrix, (self, ), 'T')
        out._backward = _backward

        return out
    
    def __pow__(self, N):
        assert isinstance(N, int | float), "Can only power up by scalars!"
        out_matrix = self.matrix ** N

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += N * (self.matrix ** (N-1)) * out.grad
        
        out = tensor(out_matrix, _children=(self, ), _operation="**")
        out._backward = _backward
        return out
    
    def __div__(self, other):
        other = self.checkOther(other)
        return self * (other**-1)
    
    def __rdiv__(self, other):
        return other * (self**-1)
    
    def sum(self):
        out_matrix = np.array(([[self.matrix.sum()]]))

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += np.ones_like(self.matrix) * out.grad

        out = tensor(out_matrix, _children=(self, ), _operation='sum()')
        out._backward = _backward
        return out

    def mean(self):
        N = self.rows * self.columns
        out_matrix = np.array(([[self.matrix.sum()/(N)]]))

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += np.ones_like(self.matrix) * out.grad / N

        out = tensor(out_matrix, _children=(self, ), _operation='mean()')
        out._backward = _backward
        return out

    def checkOther(self, other):
        if isinstance(other, int | float):
            other = tensor.const(self.rows, self.columns, other)
        elif not isinstance(other, tensor):
            other = tensor(other)
        assert other.shape() == self.shape(), "Operand Tensor sizes dont match"

        return other
    
    def zero_grad(self):
        self.grad = None
        
    def backward(self):
        self.grad = np.ones_like(self.matrix)
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        for current in reversed(topo):

            current._backward()

In [14]:
a = tensor([[2.0, 3.0]])
b = tensor([[1.0, 4.0]])
c = a + b
d = c * a
d.backward()
print(a.grad)

[[ 5. 10.]]
